In [16]:
# Import Required Libraries
import json
import pandas as pd
import os

def excel_to_assets_json(excel_file: str, json_file: str = None) -> dict:
    """
    Read an Excel file and produce a JSON file capturing the asset database
    grouped by Area -> Asset -> {land_price, house_price, rent: {...}}.

    Returns the generated assets dictionary and saves to JSON file.
    """

    # Read all sheets and take the first sheet
    excel_data = pd.read_excel(excel_file, sheet_name=None)
    df = list(excel_data.values())[0].copy()

    # Drop columns that are entirely NaN
    df = df.dropna(axis=1, how='all')

    # If the first row contains header names like 'Area' and 'Asset', use it as header
    first_row = df.iloc[0].astype(str).str.strip()
    if {'Area', 'Asset'}.issubset(set(first_row.values)):
        df.columns = first_row
        df = df.drop(df.index[0]).reset_index(drop=True)

    # Replace empty strings with NA and drop rows that are entirely NA
    df = df.replace(r'^\s*$', pd.NA, regex=True).dropna(axis=0, how='all').reset_index(drop=True)

    # Strip whitespace from object columns
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()

    # Convert known numeric columns to integers where possible
    numeric_cols = ['Land price', 'House price', 'Rent: 0', 'Rent: 1', 'Rent: 2', 'Rent: 3', 'Rent: 4']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Select only the required columns for asset structure
    required_columns = ['Area', 'Asset', 'Land price', 'House price', 'Rent: 0', 'Rent: 1', 'Rent: 2', 'Rent: 3', 'Rent: 4']
    existing_cols = [col for col in required_columns if col in df.columns]
    df = df[existing_cols].copy()

    # Drop rows with any NaN values in the selected columns and reset index
    df = df.dropna(how='any').reset_index(drop=True)

    # Build the assets JSON from the cleaned dataframe with Area as primary key
    assets = {}
    for _, row in df.iterrows():
        area = str(row['Area']).strip()
        asset_name = str(row['Asset']).strip()
        if area not in assets:
            assets[area] = {}
        assets[area][asset_name] = {
            "land_price": int(row['Land price']),
            "house_price": int(row['House price']),
            "rent": {
                "no_houses": int(row['Rent: 0']),
                "one_house": int(row['Rent: 1']),
                "two_houses": int(row['Rent: 2']),
                "three_houses": int(row['Rent: 3']),
                "four_houses": int(row['Rent: 4'])
            }
        }

    # Determine JSON filename if not provided and write file
    if json_file is None:
        json_file = os.path.splitext(excel_file)[0] + '.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(assets, f, indent=2)


def assign_asset_to_player(player_database_file: str = "Player_database.json", *, player_name: str, asset_name: str, houses: int) -> dict:
    """
    Assign an asset with a specified number of houses to a player.
    If the player already owns the asset, add the houses to the existing count.
    Houses are capped at a maximum of 4 per asset.
    
    Args:
        player_database_file: Path to JSON file containing the player database.
                             Defaults to "Player_database.json".
                             If file doesn't exist, an empty database is initialized.
        player_name: Name of the player
        asset_name: Name of the asset to assign
        houses: Number of houses to add to the asset (must be >= 0)
    
    Returns:
        Updated player database dictionary after the assignment
        
    Note:
        - If the asset is already assigned to another player, an error is printed
          and the assignment is NOT made.
        - If the asset is already owned by the same player, the houses are ADDED
          to the existing count.
        - If total houses exceed 4, a warning is printed and the total is capped at 4.
        - The updated database is saved back to the JSON file.
    """
    
    # Load existing player database or initialize empty one
    if os.path.exists(player_database_file):
        with open(player_database_file, 'r', encoding='utf-8') as f:
            player_database = json.load(f)
        print(f"INFO: Loaded player database from '{player_database_file}'")
    else:
        player_database = {}
        print(f"INFO: Created new player database (file '{player_database_file}' did not exist)")
    
    # Validate houses input
    if not isinstance(houses, int) or houses < 0:
        raise ValueError("Houses must be a non-negative integer")
    
    # Check if asset is already assigned to another player
    for other_player in player_database:
        if other_player != player_name:
            if asset_name in player_database[other_player]:
                print(f"ERROR: Asset '{asset_name}' is already assigned to player '{other_player}'. Assignment to '{player_name}' DENIED.")
                return player_database
    
    # Create player entry if doesn't exist
    if player_name not in player_database:
        player_database[player_name] = {}
    
    # Check if same player already owns this asset
    if asset_name in player_database[player_name]:
        existing_houses = player_database[player_name][asset_name]['houses']
        new_total = existing_houses + houses
        
        # Check if total exceeds 4 and cap it
        if new_total > 4:
            print(f"WARNING: Adding {houses} houses to asset '{asset_name}' for player '{player_name}' would result in {new_total} houses, but maximum is 4. Capping at 4.")
            new_total = 4
        
        print(f"INFO: Adding {houses} houses to asset '{asset_name}' for player '{player_name}': {existing_houses} + {houses} = {new_total} houses")
        player_database[player_name][asset_name] = {"houses": new_total}
    else:
        # Check if initial assignment exceeds 4
        if houses > 4:
            print(f"WARNING: Assigning {houses} houses to asset '{asset_name}' exceeds maximum of 4. Capping at 4.")
            houses = 4
        
        print(f"SUCCESS: Asset '{asset_name}' assigned to player '{player_name}' with {houses} houses")
        player_database[player_name][asset_name] = {"houses": houses}
    
    # Save updated database back to file
    with open(player_database_file, 'w', encoding='utf-8') as f:
        json.dump(player_database, f, indent=2)
    print(f"INFO: Player database saved to '{player_database_file}'")
    

In [18]:
excel_to_assets_json("Asset_database.xlsx")

In [15]:
# Test Case: Attempting to assign an asset that's already owned by another player
print("=== Test Case: Asset Conflict Detection ===\n")

# Define test database file path
test_db_file = "Player_database.json"

# Remove file if it exists from previous test run
if os.path.exists(test_db_file):
    os.remove(test_db_file)
    print(f"Removed old test database file: {test_db_file}\n")

print("Step 1: Assigning assets to Player 1")
print("-" * 50)
assign_asset_to_player(player_name="Player 1", asset_name="Boardwalk", houses=4)
assign_asset_to_player(player_name="Player 1", asset_name="Park Place", houses=3)

print("\nStep 2: Assigning assets to Player 2")
print("-" * 50)
assign_asset_to_player(player_name="Player 2", asset_name="Broadway", houses=1)
assign_asset_to_player(player_name="Player 2", asset_name="Fifth Avenue", houses=0)

# Load and display current database state
with open(test_db_file, 'r', encoding='utf-8') as f:
    test_db = json.load(f)

print("\nCurrent database state:")
print(json.dumps(test_db, indent=2))

print("\n" + "="*50)
print("Step 3: Testing Conflict - Try to assign Boardwalk to Player 2")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 2", asset_name="Boardwalk", houses=2)

print("\n" + "="*50)
print("Step 4: Testing Conflict - Try to assign Broadway to Player 1")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 1", asset_name="Broadway", houses=5)

print("\n" + "="*50)
print("Step 5: Testing House Addition - Add houses to existing asset (Player 1)")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 1", asset_name="Boardwalk", houses=2)

print("\n" + "="*50)
print("Step 6: Testing House Addition - Add more houses to same asset (Player 2)")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 2", asset_name="Broadway", houses=3)

print("\n" + "="*50)
print("Step 7: Testing House Addition - Add houses to asset with 0 houses (Player 2)")
print("-" * 50)
assign_asset_to_player(test_db_file, player_name="Player 2", asset_name="Fifth Avenue", houses=2)

# Load and display final database state
with open(test_db_file, 'r', encoding='utf-8') as f:
    test_db = json.load(f)

print("\n" + "="*50)
print("Final database state (with accumulated houses):")
print(json.dumps(test_db, indent=2))

# Verify no duplicates exist
print("\n" + "="*50)
print("Verification: Checking for duplicate assignments")
print("-" * 50)
all_assets = set()
has_duplicates = False
for player, assets in test_db.items():
    for asset in assets:
        if asset in all_assets:
            print(f"DUPLICATE FOUND: {asset} is assigned to multiple players!")
            has_duplicates = True
        else:
            all_assets.add(asset)

if not has_duplicates:
    print("✓ No duplicate assignments detected. All assets are uniquely assigned.")
else:
    print("✗ Duplicate assignments detected!")

=== Test Case: Asset Conflict Detection ===

Step 1: Assigning assets to Player 1
--------------------------------------------------
INFO: Created new player database (file 'Player_database.json' did not exist)
SUCCESS: Asset 'Boardwalk' assigned to player 'Player 1' with 4 houses
INFO: Player database saved to 'Player_database.json'
INFO: Loaded player database from 'Player_database.json'
SUCCESS: Asset 'Park Place' assigned to player 'Player 1' with 3 houses
INFO: Player database saved to 'Player_database.json'

Step 2: Assigning assets to Player 2
--------------------------------------------------
INFO: Loaded player database from 'Player_database.json'
SUCCESS: Asset 'Broadway' assigned to player 'Player 2' with 1 houses
INFO: Player database saved to 'Player_database.json'
INFO: Loaded player database from 'Player_database.json'
SUCCESS: Asset 'Fifth Avenue' assigned to player 'Player 2' with 0 houses
INFO: Player database saved to 'Player_database.json'

Current database state:
{